# IT Academy - Data Science amb Python
## Tasca 7: Estructura de control

###  [Github Registre de Logs](https://github.com/jesussantana/Registre_de_logs)

###  Exercici 1
- Normalitza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

In [3]:
import numpy as np
import pandas as pd

from io import StringIO
import re


path = '../Data/Web_access_log-akumenius.com.txt'

df = pd.read_csv(path, sep='\s\[', names =['Wev','IP','Time', 'Method', 'Status', 'Response', 'Browser', 'System'], engine='python')

df.head()

,Wev,IP,Time,Method,Status,Response,Browser,System
0,localhost 127.0.0.1 - -,"23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1....",None,NaN,NaN,NaN,NaN,NaN
1,localhost 127.0.0.1 - -,"23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1....",None,NaN,NaN,NaN,NaN,NaN
2,localhost 127.0.0.1 - -,"23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1....",None,NaN,NaN,NaN,NaN,NaN
3,localhost 127.0.0.1 - -,"23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1....",None,NaN,NaN,NaN,NaN,NaN
4,localhost 127.0.0.1 - -,"23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1....",None,NaN,NaN,NaN,NaN,NaN


###  Exercici 2
- Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

###  Exercici 3
- Geolocalitza les IP's. Aqui tens una pagina de interes:
  - [IP2Locattion](https://blog.ip2location.com/knowledge-base/how-to-add-a-sub-account-in-ip2location/)

###  Exercici 4
- Mostreu-me la teva creativitat, Sorprèn-me fes un pas més enllà amb el analysis anterior.